# 第五章 基于文档的问答

 - [一、向量存储组件](#一、向量存储组件)
     - [1.1 创建向量存储](#1.1-创建向量存储)
     - [1.2 查询创建的向量存储](#1.2-查询创建的向量存储)
 - [二、 结合表征模型和向量存储](#二、结合表征模型和向量存储)
     - [2.1 基于向量表征创建向量存储](#2.1-基于向量表征创建向量存储 )
     - [2.2 使用向量储存回答文档的相关问题](#2.2-使用向量储存回答文档的相关问题 )


常见的应用场景是使用大语言模型来构建一个能够回答关于给定文档和文档集合的问答系统。对于给定的文档, 比如从PDF、网页、公司主页中提取构建的内部文档集合，我们可以使用大语言模型来回答关于这些文档内容的问题，以帮助用户更有效地获取和使用他们所需要的信息。这种方式非常有效且灵活地适用于实际应用场景，因为它不仅仅利用大语言模型已有的训练集数据信息，它还能使用外部信息。

这个过程，我们会涉及LongChain中的其他组件，比如：表征模型（Embedding Models)和向量储存(Vector Stores)

## 一、向量存储组件

### 1.1 创建向量存储

In [11]:
import os
from langchain_openai import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain_openai.embeddings import OpenAIEmbeddings
from IPython.display import display, Markdown #在jupyter显示信息的工具
from langchain_community.vectorstores.docarray.in_memory import DocArrayInMemorySearch
from langchain.indexes.vectorstore import VectorstoreIndexCreator


file = 'data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

API_KEY = os.environ.get("CHAT_ANYWHERE_API_KEY")
BASE_URL = "https://api.chatanywhere.com.cn/v1"
embeddings = OpenAIEmbeddings(openai_api_base=BASE_URL,
                              openai_api_key=API_KEY,
                              model= "text-embedding-ada-002") #初始化

# 创建指定向量存储类，并通过文档加载器列表加载数据
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch,
                                embedding=embeddings)\
    .from_loaders([loader])
index

VectorStoreIndexWrapper(vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x1418bbf40>)

### 1.2 查询创建的向量存储

In [12]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."
from langchain.vectorstores import DocArrayInMemorySearch #向量存储(内存)
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器
llm = ChatOpenAI(base_url=BASE_URL, api_key=API_KEY, model="gpt-3.5-turbo-0301", temperature=0.0)
response = index.query(query, llm=llm)#使用索引查询创建一个响应，并传入这个查询
print(response)
display(Markdown(response))#查看查询返回的内容

| Shirt Number | Name | Description |
| --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets. |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | Rated to UPF 50+ and offers sun protection. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | Built-in UPF 50+ has the lightweight feel you want and the coverage you need when the air is hot and the UV rays are strong. Made with 71% Nylon, 29% Polyester. Additional features include wrinkle resistance, front and back cape venting, and two front bellows pockets. |
| 255 | Sun Shield Shirt | High-performance sun shirt is guaranteed to protect from harmful UV rays. Made wit

| Shirt Number | Name | Description |
| --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets. |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | Rated to UPF 50+ and offers sun protection. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | Built-in UPF 50+ has the lightweight feel you want and the coverage you need when the air is hot and the UV rays are strong. Made with 71% Nylon, 29% Polyester. Additional features include wrinkle resistance, front and back cape venting, and two front bellows pockets. |
| 255 | Sun Shield Shirt | High-performance sun shirt is guaranteed to protect from harmful UV rays. Made with 78% nylon, 22% Lycra Xtra Life fiber. Additional features include moisture-wicking, abrasion resistance, and fits comfortably over your favorite swimsuit. |

The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting that lets in cool breezes and two front bellows pockets. The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon, and has front and back cape venting, and two front bellows pockets. The Men's TropicVibe Shirt, Short-Sleeve has built-in UPF 50+ and is made with 71% Nylon, 29% Polyester. It has wrinkle resistance, front and back cape venting, and two front bellows pockets. The Sun Shield Shirt is made with 78% nylon, 22% Lycra Xtra Life fiber, and is moisture-wicking and abrasion-resistant. It fits comfortably over your favorite swimsuit. All shirts have UPF 50+ sun protection, blocking 98% of the sun's harmful rays.

得到了一个Markdown表格，其中包含所有带有防晒衣的衬衫的名称和描述，还得到了一个语言模型提供的不错的小总结

## 二、 结合表征模型和向量存储

使用语言模型与文档结合使用时，语言模型一次只能使用几千个单词的信息。如果我们文档比较长，如何让语言模型回答关于其中所有内容的问题呢？我们通过几个通过向量表征（Embeddings）和向量存储(Vector Store)实现

文本表征（Embeddings)是对文本语义的向量表征，相似内容的文本具有相似的表征向量。这使我们可以在向量空间中比较文本的相似性。

向量数据库(Vector Database)用来存储文档的文本块。对于给定的文档，我们首先将其分成较小的文本块（chunks），然后获取每个小文本块的文本表征，并将这些表征储存在向量数据库中。这个流程正是创建索引(index)的过程。将文档分成小文本块的原因在于我们可能无法将整个文档传入语言模型进行处理。

索引创建完成后，我们可以使用索引来查找与传入的查询(Query)最相关的文本片段 - 首先我们为查询获取向量表征，然后我们将其与向量数据库中的所有向量进行比较并选择最相似的n个文本块，最后我们将这些相似的文本块构建提示，并输入到语言模型，从而得到最终答案。

### 2.1 基于向量表征创建向量存储

In [3]:
import os
from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
from langchain_openai import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain_openai.embeddings import OpenAIEmbeddings
from IPython.display import display, Markdown #在jupyter显示信息的工具
from langchain_community.vectorstores.docarray.in_memory import DocArrayInMemorySearch
from langchain_core.documents.base import Document
from typing import List


file = 'data/OutdoorClothingCatalog_1000.csv'
API_KEY = os.environ.get("CHAT_ANYWHERE_API_KEY")
BASE_URL = "https://api.chatanywhere.com.cn/v1"

#创建一个文档加载器，通过csv格式加载
loader = CSVLoader(file_path=file)
docs = loader.load()
print(docs[0])

# 向量
embeddings = OpenAIEmbeddings(openai_api_base=BASE_URL,
                              openai_api_key=API_KEY,
                              model= "text-embedding-ada-002") #初始化
print(type(embeddings))

#因为文档比较短了，所以这里不需要进行任何分块,可以直接进行向量表征
#使用初始化OpenAIEmbedding实例上的查询方法embed_query为文本创建向量表征
embed = embeddings.embed_query("Hi my name is Harrison")
print(len(embed))
print(embed[:5])

# 基于向量表征创建向量存储
db = DocArrayInMemorySearch.from_documents(docs, embeddings)
db

page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries." metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 0}
<class 'langchain_openai.embeddings.base.OpenAIEmbeddings'>
1536
[-0.0219351164996624, 0.006751196924597025, -0.0182583499699831, -0

### 2.2 使用向量储存回答文档的相关问题

我们可以看到一个返回了四个结果。输出的第一结果是一件关于防晒的衬衫，满足我们查询的要求：`请推荐一件防晒功能的衬衫（Please suggest a shirt with sunblocking）`

In [8]:
query = "Please suggest a shirt with sunblocking"

#使用上面的向量存储来查找与传入查询类似的文本，得到一个相似文档列表
docs: List[Document] = db.similarity_search(query)

print(len(docs))
print(docs[0])

#导入大语言模型
llm = ChatOpenAI(base_url=BASE_URL, api_key=API_KEY, model="gpt-3.5-turbo-0301", temperature=0.0)

#合并获得的相似文档内容
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

#将合并的相似文档内容后加上问题（question）输入到 `llm.call_as_llm`中
#这里问题是：以Markdown表格的方式列出所有具有防晒功能的衬衫并总结
response = llm.invoke(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

display(Markdown(response.content))

4
page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.' metadata={'source': 'data/OutdoorClothingCatalog_1000.csv', 'row': 255}


| Name | Description |
| --- | --- |
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ sun protection, moisture-wicking, and abrasion-resistant fabric. Fits comfortably over swimsuits. Recommended by The Skin Cancer Foundation. |
| Men's Plaid Tropic Shirt | Ultracomfortable shirt with UPF 50+ sun protection, wrinkle-free fabric, and front/back cape venting. Made with 52% polyester and 48% nylon. |
| Men's TropicVibe Shirt | Men's sun-protection shirt with built-in UPF 50+ and front/back cape venting. Made with 71% nylon and 29% polyester. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with UPF 50+ sun protection, front/back cape venting, and two front bellows pockets. Made with 100% polyester and is wrinkle-resistant. |

All of these shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. They are made with high-performance fabrics that are moisture-wicking, abrasion-resistant, and/or wrinkle-free. Some have front/back cape venting for added comfort in hot weather. The Sun Shield Shirt is recommended by The Skin Cancer Foundation.

在此处打印响应，我们可以看到我们得到了一个表格，正如我们所要求的那样

In [9]:
''' 
通过LangChain链封装起来
创建一个检索QA链，对检索到的文档进行问题回答，要创建这样的链，我们将传入几个不同的东西
1、语言模型，在最后进行文本生成
2、传入链类型，这里使用stuff，将所有文档塞入上下文并对语言模型进行一次调用
3、传入一个检索器
'''
#基于向量储存，创建检索器
retriever = db.as_retriever()
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

query =  "Please suggest a shirt with sunblocking. Please list all your shirts with sun protection in a table in markdown and summarize each one."#创建一个查询并在此查询上运行链

response = qa_stuff.invoke(query)
response



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Please suggest a shirt with sunblocking. Please list all your shirts with sun protection in a table in markdown and summarize each one.',
 'result': "| Shirt Name | Description |\n| --- | --- |\n| Sun Shield Shirt | High-performance sun shirt with UPF 50+ sun protection, blocking 98% of the sun's harmful rays. Made of 78% nylon and 22% Lycra Xtra Life fiber. Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion-resistant for season after season of wear. |\n| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets. |\n| Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable sun protection rated to UPF 50+. Made with 52% polyester and 48% nylon. SunSmart technology blocks 98% of the sun's harmful UV rays. Wrinkle-free and q

In [10]:
display(Markdown(response['result'])) #使用 display 和 markdown 显示它

| Shirt Name | Description |
| --- | --- |
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ sun protection, blocking 98% of the sun's harmful rays. Made of 78% nylon and 22% Lycra Xtra Life fiber. Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion-resistant for season after season of wear. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable sun protection rated to UPF 50+. Made with 52% polyester and 48% nylon. SunSmart technology blocks 98% of the sun's harmful UV rays. Wrinkle-free and quickly evaporates perspiration. With front and back cape venting and two front bellows pockets. |
| Men's TropicVibe Shirt, Short-Sleeve | Men’s sun-protection shirt with built-in UPF 50+. Made of 71% Nylon and 29% Polyester. Wrinkle-resistant with front and back cape venting that lets in cool breezes and two front bellows pockets. |

All of the listed shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Sun Shield Shirt is made of 78% nylon and 22% Lycra Xtra Life fiber, while the Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester. The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon, and the Men's TropicVibe Shirt, Short-Sleeve is made of 71% Nylon and 29% Polyester. All shirts are machine washable and dryable, and have additional features such as front and back cape venting and two front bellows pockets.

这两个方式返回相同的结果

想在许多不同类型的块上执行相同类型的问答，该怎么办？之前的实验中只返回了4个文档，如果有多个文档，那么我们可以使用几种不同的方法
* Map Reduce   
将所有块与问题一起传递给语言模型，获取回复，使用另一个语言模型调用将所有单独的回复总结成最终答案，它可以在任意数量的文档上运行。可以并行处理单个问题，同时也需要更多的调用。它将所有文档视为独立的
* Refine    
用于循环许多文档，际上是迭代的，建立在先前文档的答案之上，非常适合前后因果信息并随时间逐步构建答案，依赖于先前调用的结果。它通常需要更长的时间，并且基本上需要与Map Reduce一样多的调用
* Map Re-rank   
对每个文档进行单个语言模型调用，要求它返回一个分数，选择最高分，这依赖于语言模型知道分数应该是什么，需要告诉它，如果它与文档相关，则应该是高分，并在那里精细调整说明，可以批量处理它们相对较快，但是更加昂贵
* Stuff    
将所有内容组合成一个文档